In [1]:
from IMPA.dataset.data_loader import CellDataLoader
from IMPA.solver import IMPAmodule
from omegaconf import OmegaConf
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import torch

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
class Args:
    def __init__(self, dictionary):
        self.__dict__ = dictionary

    def __getattr__(self, key):
        if key in self.__dict__:
            return self.__dict__[key]
        else:
            raise AttributeError(f"'DictToObject' object has no attribute '{key}'")

    def __call__(self, key):
        return self.__getattr__(key)

def t2np(t):
    return ((t.permute(0,2, 3, 1) + 1) / 2).clamp(0,1).cpu().numpy()

In [16]:
config_dict = {
    "task_name": "cpg0000_total_large_transported",
    "img_size": 96,
    "latent_dim": 100,
    "hidden_dim": 512,
    "style_dim": 32,
    "stochastic": True,
    "z_dimension": 20,
    "dim_in": 64,
    "lambda_reg": 1,
    "lambda_cyc": 1,
    "lambda_sty": 1,
    "lambda_ds": 1,
    "total_epochs": 100,
    "ds_iter": 200000,
    "resume_iter": 0,
    "batch_size": 32,
    "val_batch_size": 20,
    "lr": 0.0001,
    "f_lr": 0.0001,
    "beta1": 0,
    "beta2": 0.99,
    "weight_decay": 0.0001,
    "num_outs_per_domain": 10,
    "single_style": True,
    "ood_set": None,
    "mol_list": None,
    "balanced": False,
    "trainable_emb": True,
    "dataset_name": "cpg0000",
    "n_channels": 5,
    "num_workers": 6,
    "seed": 42,
    "image_path": "/lustre/groups/ml01/datasets/projects/cpg0000_alessandro/cpg0000_u2os_normalized_segmented_large_transported",
    "data_index_path": "/lustre/groups/ml01/datasets/projects/cpg0000_alessandro/metadata/metadata_large.csv",
    "embedding_path": None,
    "experiment_directory": "../project_folder/experiments/",
    "sample_dir": "sample",
    "checkpoint_dir": "checkpoint",
    "basal_vs_real_folder": "basal_vs_real",
    "naming_key": "dataset_name",
    "embedding_folder": "embeddings",
    "resume_dir": '',
    "augment_train": True,
    "normalize": True,
    "print_every": 10,
    "sample_every": 1000,
    "save_every": 500,
    "eval_every": 500,
    "encode_rdkit": True,
    "num_layers_mapping_net": 1,
    "filename": "epoch_{epoch:04d}",
    "monitor": "fid_transformations",
    "mode": "min",
    "save_last": True,
    "offline": False,
    "project": "cpg0000_total_large_transported",
    "log_model": True,
    "accelerator": "gpu",
    "log_every_n_steps": 10, 
    "add_controls": False,
    "multimodal": False,
    "modality_list": None, 
    "use_condition_embedding": False
}

args = OmegaConf.create(config_dict)

dest_dir = "/home/icb/alessandro.palma/environment/IMPA/IMPA/project_folder/experiments/20240115_35929b66-de4e-4acf-a58f-d95b26034998_cpg0000_total_large_transported"

In [17]:
dataloader = CellDataLoader(args)

100


In [18]:
solver = IMPAmodule(args, dest_dir, dataloader)
solver._load_checkpoint(76)

Number of parameters in generator: 24102789
Number of parameters in style_encoder: 14314208
Number of parameters in discriminator: 14681003
Number of parameters in mapping_network: 3872
Initializing embedding_matrix...
Initializing generator...
Initializing style_encoder...
Initializing discriminator...
Initializing mapping_network...
IMPAmodule(
  (embedding_matrix): Embedding(747, 100)
  (generator): DataParallel(
    (module): Generator(
      (from_rgb): Conv2d(5, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (encode): ModuleList(
        (0): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False

RuntimeError: Error(s) in loading state_dict for Discriminator:
	Missing key(s) in state_dict: "conv_blocks.0.weight", "conv_blocks.0.bias", "conv_blocks.1.conv1.weight", "conv_blocks.1.conv1.bias", "conv_blocks.1.conv2.weight", "conv_blocks.1.conv2.bias", "conv_blocks.1.conv1x1.weight", "conv_blocks.2.conv1.weight", "conv_blocks.2.conv1.bias", "conv_blocks.2.conv2.weight", "conv_blocks.2.conv2.bias", "conv_blocks.2.conv1x1.weight", "conv_blocks.3.conv1.weight", "conv_blocks.3.conv1.bias", "conv_blocks.3.conv2.weight", "conv_blocks.3.conv2.bias", "conv_blocks.3.conv1x1.weight", "conv_blocks.4.conv1.weight", "conv_blocks.4.conv1.bias", "conv_blocks.4.conv2.weight", "conv_blocks.4.conv2.bias", "conv_blocks.5.conv1.weight", "conv_blocks.5.conv1.bias", "conv_blocks.5.conv2.weight", "conv_blocks.5.conv2.bias", "conv_blocks.7.weight", "conv_blocks.7.bias", "head.weight", "head.bias". 
	Unexpected key(s) in state_dict: "conv.0.weight", "conv.0.bias", "conv.1.conv1.weight", "conv.1.conv1.bias"

In [ ]:
train_dataloader = dataloader.train_dataloader()
val_dataloader = dataloader.val_dataloader()

**Check a bunch of drugs**

In [9]:
pert_to_inspect = ["1-octanol", "7-hydroxystaurosporine", "AVL-292", "AZD7762", "BI-2536", "BMS-536924", 
                      "BVT-948", "GK921", "GSK1070916", "GSK2110183", "ITGB2", "KG-5", "NSC-663284", 
                      "NSC-95397", "PP-121", "RGB-286638", "SU3327", "TG-02", "TG-101348", "UNC1999", 
                      "UNC2025", "anisomycin", "azeliragon", "briciclib", "colchicine", "delanzomib", 
                      "dexamethasone", "homoharringtonine", "ibutamoren", "ixabepilone", "oxibendazole",
                      "ponatinib", "puromycin", "pyrrolidine-dithiocarbamate", "romidepsin", "ryuvidine"
                  ]

In [ ]:
transf_images = {}
real_images = {}

# z_common = torch.randn(1, 100, args.z_dimension).mean(1).cuda()
with torch.no_grad():
    for batch in train_dataloader:
        X_ctr = batch["X"][0]
        for pert in tqdm(pert_to_inspect):
            # z = z_common.repeat(X_ctr.shape[0], 1)
            z = torch.randn(X_ctr.shape[0], args.z_dimension).cuda()
            id_pert = dataloader.mol2id[pert]*torch.ones(X_ctr.shape[0]).long().cuda()
            y = solver.embedding_matrix(id_pert)
            y = torch.cat([y, z], dim=1)
            y = solver.nets.mapping_network(y)
            _, X_generated = solver.nets.generator(X_ctr, y)
            transf_images[pert] = t2np(X_generated.detach().cpu())
            real_images[pert] = t2np(X_ctr.detach().cpu())
        break

In [ ]:
for pert in transf_images:
    print(pert)
    for i in range(len(batch)):
        print("real")
        plt.figure(figsize=(1,1))
        plt.imshow(real_images[pert][i][:,:,[0,3,4]])
        plt.show()
        print("transf")
        plt.figure(figsize=(1,1))
        plt.imshow(transf_images[pert][i][:,:,[0,3,4]])
        plt.show()        